In [ ]:
import imageio
import torch
import sys
import numpy as np
import torchvision.transforms as transforms

from glob import glob
from algorithm.SRNet import Srnet
from utils.helper import *


cfg = load_config()
device = torch.device(cfg.device)
run_folder = create_folder(cfg.run_folder)

# to be reproducible
if torch.cuda.is_available():
    cudnn.benchmark = True
    if cfg.seed is not None:
        np.random.seed(cfg.seed)  # Numpy module.
        random.seed(cfg.seed)  # Python random module.
        torch.manual_seed(cfg.seed)  # Sets the seed for generating random numbers.
        torch.cuda.manual_seed(cfg.seed)  # Sets the seed for generating random numbers for the current GPU.
        torch.cuda.manual_seed_all(cfg.seed)  # Sets the seed for generating random numbers on all GPUs.
        cudnn.deterministic = True

# Print the configuration.
logging.basicConfig(level=logging.INFO, format='%(message)s', handlers=[logging.FileHandler(os.path.join(run_folder, f'run.log')), logging.StreamHandler(sys.stdout)])
logging.info("Experiment Configuration:")
logging.info("CUDA_VISIBLE_DEVICES：{}".format(os.getenv('CUDA_VISIBLE_DEVICES')))
logging.info(cfg)
logging.info("run_folder:{}".format(run_folder))

TEST_BATCH_SIZE = cfg.batch_size
COVER_PATH = cfg.cover_path
STEGO_PATH = cfg.stego_path
CHKPT = cfg.ckp

cover_image_names = glob(COVER_PATH)
stego_image_names = glob(STEGO_PATH)

cover_labels = np.zeros((len(cover_image_names)))
stego_labels = np.ones((len(stego_image_names)))

model = Srnet().to(device)

ckpt = torch.load(CHKPT)
model.load_state_dict(ckpt[''])  # model weights

images = torch.empty((TEST_BATCH_SIZE, 1, cfg.resize, cfg.resize), dtype=torch.float)
test_accuracy = []

test_tf = transforms.Compose([
    ToPILImage(),
    Resize(cfg.resize, cfg.resize),
    ToGrayImage(),
    ToTensor(cfg.resize)
])

for idx in range(0, len(cover_image_names), TEST_BATCH_SIZE//2):

    cover_batch = cover_image_names[idx:idx+TEST_BATCH_SIZE//2]
    stego_batch = stego_image_names[idx:idx+TEST_BATCH_SIZE//2]

    batch = []
    batch_labels = []

    x_i = 0
    y_i = 0

    for i in range(2*len(cover_batch)):
        if i % 2 == 0:
            batch.append(stego_batch[x_i])
            batch_labels.append(1)
            x_i += 1
        else:
            batch.append(cover_batch[y_i])
            batch_labels.append(0)
            y_i += 1

    for i in range(0, TEST_BATCH_SIZE, 2):
        sample = test_tf({'cover': imageio.imread(batch[i]), 'stego': imageio.imread(batch[i+1])})
        images[i, 0, :, :], images[i+1, 0, :, :] = sample['cover'], sample['stego']

    image_tensor = images.cuda()
    batch_labels = torch.tensor(batch_labels, dtype=torch.long).cuda()

    outputs = model(image_tensor)
    prediction = outputs.data.max(1)[1]

    accuracy = prediction.eq(batch_labels.data).sum()*100.0/(batch_labels.size()[0])
    test_accuracy.append(accuracy.item())

logging.info("detection accuracy = %.2f"%(sum(test_accuracy)/len(test_accuracy)))